In [104]:
import time
import math
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
import numpy as np
import pandas as pd
import sklearn.preprocessing as prep

In [105]:
df = pd.read_csv('^NSEI_withoutnull.csv')
df.head()

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date,Open,High,Low,Close,Future_Close
0,2007-11-22,5564.649902,5608.649902,5394.350098,5519.350098,5608.600098
1,2007-11-23,5524.299805,5638.600098,5514.750000,5608.600098,5731.700195
2,2007-11-26,5611.299805,5772.549805,5608.950195,5731.700195,5698.149902
3,2007-11-27,5729.250000,5743.549805,5655.600098,5698.149902,5617.549805
4,2007-11-28,5699.549805,5749.950195,5595.500000,5617.549805,5634.600098


In [106]:
df = df.dropna(axis=0, how='any')
df.shape

(2418, 6)

In [107]:
df = df.drop(['Date'], axis=1)
df.head()

,Open,High,Low,Close,Future_Close
0,5564.649902,5608.649902,5394.350098,5519.350098,5608.600098
1,5524.299805,5638.600098,5514.750000,5608.600098,5731.700195
2,5611.299805,5772.549805,5608.950195,5731.700195,5698.149902
3,5729.250000,5743.549805,5655.600098,5698.149902,5617.549805
4,5699.549805,5749.950195,5595.500000,5617.549805,5634.600098


In [108]:
df['Future_Close'] = df[' Future_Close'] 
df.head()

,Open,High,Low,Close,Future_Close,Future_Close
0,5564.649902,5608.649902,5394.350098,5519.350098,5608.600098,5608.600098
1,5524.299805,5638.600098,5514.750000,5608.600098,5731.700195,5731.700195
2,5611.299805,5772.549805,5608.950195,5731.700195,5698.149902,5698.149902
3,5729.250000,5743.549805,5655.600098,5698.149902,5617.549805,5617.549805
4,5699.549805,5749.950195,5595.500000,5617.549805,5634.600098,5634.600098


In [109]:
df = df.drop([' Future_Close'], axis=1)
df.head()

,Open,High,Low,Close,Future_Close
0,5564.649902,5608.649902,5394.350098,5519.350098,5608.600098
1,5524.299805,5638.600098,5514.750000,5608.600098,5731.700195
2,5611.299805,5772.549805,5608.950195,5731.700195,5698.149902
3,5729.250000,5743.549805,5655.600098,5698.149902,5617.549805
4,5699.549805,5749.950195,5595.500000,5617.549805,5634.600098


### New Column 'Rat' added = Future_Close/Current_Close

In [110]:
df['Rat'] = df['Future_Close']/df['Close']

In [111]:
df['Rat']

0       1.016170
1       1.021948
2       0.994147
3       0.985855
4       1.003035
5       1.022743
6       1.017743
7       0.998866
8       1.013937
9       1.002475
10      1.003291
11      0.997707
12      1.022926
13      1.010177
14      0.983570
15      0.998283
16      0.955239
17      0.993993
18      1.001541
19      1.002669
20      1.037909
21      1.014311
22      1.001771
23      0.999704
24      1.009688
25      1.000937
26      1.005704
27      0.999862
28      1.015497
29      1.000765
          ...   
2388    1.000907
2389    1.002823
2390    0.996790
2391    1.011177
2392    1.007037
2393    1.006236
2394    1.000352
2395    0.997694
2396    0.997454
2397    1.002244
2398    1.008587
2399    1.004706
2400    0.997994
2401    1.003933
2402    0.997264
2403    1.010179
2404    0.998400
2405    1.002753
2406    0.999933
2407    0.990274
2408    0.995459
2409    1.000563
2410    1.001242
2411    0.990622
2412    0.996249
2413    0.993271
2414    1.009557
2415    1.0067

In [112]:
df['Rat'] = df['Rat']>1
df['Rat'] = df['Rat'].astype(float)
df.head()

,Open,High,Low,Close,Future_Close,Rat
0,5564.649902,5608.649902,5394.350098,5519.350098,5608.600098,1.0
1,5524.299805,5638.600098,5514.750000,5608.600098,5731.700195,1.0
2,5611.299805,5772.549805,5608.950195,5731.700195,5698.149902,0.0
3,5729.250000,5743.549805,5655.600098,5698.149902,5617.549805,0.0
4,5699.549805,5749.950195,5595.500000,5617.549805,5634.600098,1.0


#### we're dropping future_close since there can be only one dependent variable

In [113]:
df = df.drop(['Future_Close'], axis=1)
df.head()

,Open,High,Low,Close,Rat
0,5564.649902,5608.649902,5394.350098,5519.350098,1.0
1,5524.299805,5638.600098,5514.750000,5608.600098,1.0
2,5611.299805,5772.549805,5608.950195,5731.700195,0.0
3,5729.250000,5743.549805,5655.600098,5698.149902,0.0
4,5699.549805,5749.950195,5595.500000,5617.549805,1.0


In [114]:
from keras.models import Sequential
from keras.layers import Dense

np.random.seed(7)

In [115]:
df.shape

(2418, 5)

In [116]:
df_target = pd.get_dummies(df['Rat'])
df_new = pd.concat([df, df_target], axis=1)
df_new = df_new.drop(['Rat'], axis=1)
df = df_new
df.head()

,Open,High,Low,Close,0.0,1.0
0,5564.649902,5608.649902,5394.350098,5519.350098,0,1
1,5524.299805,5638.600098,5514.750000,5608.600098,0,1
2,5611.299805,5772.549805,5608.950195,5731.700195,1,0
3,5729.250000,5743.549805,5655.600098,5698.149902,1,0
4,5699.549805,5749.950195,5595.500000,5617.549805,0,1


In [117]:
df.shape

(2418, 6)

In [118]:
train_start=0
train_end=int(np.floor(0.8*df.shape[0]))
test_start=train_end+1
test_end=int(df.shape[0])
df = df.values
data_train=df[np.arange(train_start, train_end),:]
data_test=df[np.arange(test_start,test_end),:]
data_train=pd.DataFrame(data_train)
data_test=pd.DataFrame(data_test)

In [119]:
print(data_train.shape)
data_train.head()


(1934, 6)


,0,1,2,3,4,5
0,5564.649902,5608.649902,5394.350098,5519.350098,0.0,1.0
1,5524.299805,5638.600098,5514.750000,5608.600098,0.0,1.0
2,5611.299805,5772.549805,5608.950195,5731.700195,1.0,0.0
3,5729.250000,5743.549805,5655.600098,5698.149902,1.0,0.0
4,5699.549805,5749.950195,5595.500000,5617.549805,0.0,1.0


In [120]:
print(data_test.shape)
data_test.head()

(483, 6)


,0,1,2,3,4,5
0,7958.149902,7972.149902,7934.149902,7954.899902,1.0,0.0
1,7976.700195,7979.299805,7910.799805,7931.350098,1.0,0.0
2,7902.299805,7912.299805,7853.299805,7864.149902,1.0,0.0
3,7817.600098,7821.399902,7775.700195,7781.899902,1.0,0.0
4,7816.549805,7825.399902,7746.049805,7765.399902,1.0,0.0


In [121]:
x_train=data_train.iloc[:,0:4]
y_train=data_train.iloc[:, 4:6]
x_test=data_test.iloc[:,0:4]
y_test=data_test.iloc[:, 4:6]
features = 4

In [122]:
y_train.head()

,4,5
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0


In [123]:
import tensorflow as tf
X = tf.placeholder(dtype=tf.float32, shape=[None, features])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 2])

In [124]:
n_neurons_1 = 15
n_neurons_2 = 10
n_neurons_3 = 5
n_target = 2
sigma = 1
training_epochs = 1000
display_step = 1

In [125]:
import keras
weight_initializer = keras.initializers.VarianceScaling(mode="fan_avg", distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()

In [126]:
W_hidden_1 = tf.Variable(weight_initializer([features, n_neurons_1]))
bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))
W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))
W_hidden_3 = tf.Variable(weight_initializer([n_neurons_2, n_neurons_3]))
bias_hidden_3 = tf.Variable(bias_initializer([n_neurons_3]))
W_out = tf.Variable(weight_initializer([n_neurons_3, n_target]))
bias_out = tf.Variable(bias_initializer([n_target]))
hidden_1 = tf.nn.relu(tf.add(tf.matmul(X, W_hidden_1), bias_hidden_1))
hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))
hidden_3 = tf.nn.relu(tf.add(tf.matmul(hidden_2, W_hidden_3), bias_hidden_3))
pred = (tf.add(tf.matmul(hidden_3, W_out), bias_out))

In [127]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

In [132]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(x_train)/100)
        X_batches = np.array_split(x_train, total_batch)
        Y_batches = np.array_split(y_train, total_batch)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = X_batches[i], Y_batches[i]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={X: batch_x,
                                                          Y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: x_train, Y: y_train}))
    global result 
    result = tf.argmax(pred, 1).eval({X: x_test, Y: y_test})
    print(result)

Epoch: 0001 cost= 200.986994693
Epoch: 0002 cost= 31.426694581
Epoch: 0003 cost= 9.447619539
Epoch: 0004 cost= 3.864345516
Epoch: 0005 cost= 2.536299975
Epoch: 0006 cost= 1.519815410
Epoch: 0007 cost= 1.409641122
Epoch: 0008 cost= 1.243247995
Epoch: 0009 cost= 1.316457388
Epoch: 0010 cost= 1.274087341
Epoch: 0011 cost= 1.324569611
Epoch: 0012 cost= 1.352151723
Epoch: 0013 cost= 1.375449843
Epoch: 0014 cost= 1.399618390
Epoch: 0015 cost= 1.416372265
Epoch: 0016 cost= 1.429466897
Epoch: 0017 cost= 1.439118868
Epoch: 0018 cost= 1.445679614
Epoch: 0019 cost= 1.449924620
Epoch: 0020 cost= 1.452201087
Epoch: 0021 cost= 1.452900501
Epoch: 0022 cost= 1.452370534
Epoch: 0023 cost= 1.450755568
Epoch: 0024 cost= 1.448358322
Epoch: 0025 cost= 1.445242355
Epoch: 0026 cost= 1.441578849
Epoch: 0027 cost= 1.437447297
Epoch: 0028 cost= 1.432956937
Epoch: 0029 cost= 1.428126436
Epoch: 0030 cost= 1.423084024
Epoch: 0031 cost= 1.417811071
Epoch: 0032 cost= 1.412341855
Epoch: 0033 cost= 1.406739847
Epoch: 

Epoch: 0280 cost= 0.728448400
Epoch: 0281 cost= 0.729685937
Epoch: 0282 cost= 0.727536183
Epoch: 0283 cost= 0.727776242
Epoch: 0284 cost= 0.727704694
Epoch: 0285 cost= 0.727638282
Epoch: 0286 cost= 0.727199263
Epoch: 0287 cost= 0.726123132
Epoch: 0288 cost= 0.725911250
Epoch: 0289 cost= 0.726672474
Epoch: 0290 cost= 0.725755058
Epoch: 0291 cost= 0.726057238
Epoch: 0292 cost= 0.724602906
Epoch: 0293 cost= 0.725299468
Epoch: 0294 cost= 0.724995394
Epoch: 0295 cost= 0.726549233
Epoch: 0296 cost= 0.723555279
Epoch: 0297 cost= 0.725788041
Epoch: 0298 cost= 0.725044988
Epoch: 0299 cost= 0.723581323
Epoch: 0300 cost= 0.722652689
Epoch: 0301 cost= 0.723305674
Epoch: 0302 cost= 0.722124125
Epoch: 0303 cost= 0.723164097
Epoch: 0304 cost= 0.721438690
Epoch: 0305 cost= 0.722019490
Epoch: 0306 cost= 0.721905059
Epoch: 0307 cost= 0.721880075
Epoch: 0308 cost= 0.721651206
Epoch: 0309 cost= 0.722221506
Epoch: 0310 cost= 0.724443247
Epoch: 0311 cost= 0.719981934
Epoch: 0312 cost= 0.721604513
Epoch: 031

Epoch: 0558 cost= 0.710358864
Epoch: 0559 cost= 0.710625504
Epoch: 0560 cost= 0.710205963
Epoch: 0561 cost= 0.709945155
Epoch: 0562 cost= 0.709439133
Epoch: 0563 cost= 0.709228274
Epoch: 0564 cost= 0.709123169
Epoch: 0565 cost= 0.708827449
Epoch: 0566 cost= 0.708636770
Epoch: 0567 cost= 0.708542908
Epoch: 0568 cost= 0.708271867
Epoch: 0569 cost= 0.708104152
Epoch: 0570 cost= 0.708265521
Epoch: 0571 cost= 0.708016954
Epoch: 0572 cost= 0.707918239
Epoch: 0573 cost= 0.707700642
Epoch: 0574 cost= 0.707610334
Epoch: 0575 cost= 0.707392420
Epoch: 0576 cost= 0.707165009
Epoch: 0577 cost= 0.707143106
Epoch: 0578 cost= 0.706944092
Epoch: 0579 cost= 0.706723549
Epoch: 0580 cost= 0.706708641
Epoch: 0581 cost= 0.706518804
Epoch: 0582 cost= 0.706305774
Epoch: 0583 cost= 0.706303371
Epoch: 0584 cost= 0.706118424
Epoch: 0585 cost= 0.705915617
Epoch: 0586 cost= 0.705782906
Epoch: 0587 cost= 0.705666297
Epoch: 0588 cost= 0.705547621
Epoch: 0589 cost= 0.705430363
Epoch: 0590 cost= 0.705315759
Epoch: 059

Epoch: 0833 cost= 0.697165637
Epoch: 0834 cost= 0.696913983
Epoch: 0835 cost= 0.696628589
Epoch: 0836 cost= 0.696354047
Epoch: 0837 cost= 0.696169646
Epoch: 0838 cost= 0.695952692
Epoch: 0839 cost= 0.695701549
Epoch: 0840 cost= 0.695563542
Epoch: 0841 cost= 0.695432070
Epoch: 0842 cost= 0.695170246
Epoch: 0843 cost= 0.695112919
Epoch: 0844 cost= 0.694921352
Epoch: 0845 cost= 0.694823911
Epoch: 0846 cost= 0.694692599
Epoch: 0847 cost= 0.694553083
Epoch: 0848 cost= 0.694444954
Epoch: 0849 cost= 0.694344364
Epoch: 0850 cost= 0.694210965
Epoch: 0851 cost= 0.694172624
Epoch: 0852 cost= 0.694072557
Epoch: 0853 cost= 0.693954041
Epoch: 0854 cost= 0.693902998
Epoch: 0855 cost= 0.693814914
Epoch: 0856 cost= 0.693737777
Epoch: 0857 cost= 0.693679853
Epoch: 0858 cost= 0.693621610
Epoch: 0859 cost= 0.693521867
Epoch: 0860 cost= 0.693525076
Epoch: 0861 cost= 0.693423215
Epoch: 0862 cost= 0.693397563
Epoch: 0863 cost= 0.693315189
Epoch: 0864 cost= 0.693268694
Epoch: 0865 cost= 0.693212199
Epoch: 086